In [1]:
#importing all of the libraries that will be needed
import pandas as pd 
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
%matplotlib inline 
sns.set(color_codes=True)
import statistics as stats
from IPython.display import display, HTML

In [2]:
# Inserting some images that is related to the assignment topic

# Paths to my image files
image_paths = ['11.jpg', '12.jpg', '13.jpg']

# Generate HTML code to display images side by side
images_html = ''.join([f'<img src="{path}" style="width: 310px; margin: 0px; float: left;" />' for path in image_paths])

# Display HTML code
display(HTML(images_html))

In [16]:
#Read files from the URL into the pandas DataFrame
url = "airandtravel1.csv"
travel_df = pd.read_csv(url) 

## Where did Ireland's Tourists Come From? Cleaning and Preperation
This section will aim use data preperation methods such a `.head()` , `.shape`, `describe`, `info.()` etc to clean and sort the data first

In [17]:
#Using .head() to get a sense for the data, its structure and content
travel_df.head()

,STATISTIC Label,Year,Month,Country,Direction,UNIT,VALUE
0,Air and Sea Travel,2010,January,Great Britain,Arrivals,Thousand,361.5
1,Air and Sea Travel,2010,January,Great Britain,Departures,Thousand,407.6
2,Air and Sea Travel,2010,January,Other UK (1),Arrivals,Thousand,0.9
3,Air and Sea Travel,2010,January,Other UK (1),Departures,Thousand,1.1
4,Air and Sea Travel,2010,January,Belgium,Arrivals,Thousand,12.5


In [18]:
travel_df.shape

(6084, 7)

In [21]:
#Here i am dropping any data before 2021, as I was to work with the timeframe 2021-2023
filtered_df = travel_df[travel_df['Year'] >= 2021]

# Reset index 
filtered_df.reset_index(drop=True, inplace=True)

#my df is now called filtered_df

In [22]:
filtered_df.shape

(1332, 7)

In [27]:
# This provides me with information such as the datatypes. 
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332 entries, 0 to 1331
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATISTIC Label  1332 non-null   object 
 1   Year             1332 non-null   int64  
 2   Month            1332 non-null   object 
 3   Country          1332 non-null   object 
 4   Direction        1332 non-null   object 
 5   UNIT             1332 non-null   object 
 6   VALUE            1328 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 73.0+ KB


In [28]:
filtered_df.describe()
# because only value and year is an integer type (float) I am only returned with two coulmn of data for my statistical analsis 

,Year,VALUE
count,1332.000000,1328.000000
mean,2022.054054,132.897816
std,0.868563,321.662704
min,2021.000000,0.000000
25%,2021.000000,11.075000
50%,2022.000000,34.600000
75%,2023.000000,91.125000
max,2024.000000,2270.700000


In [29]:
filtered_df.isnull().sum()

STATISTIC Label    0
Year               0
Month              0
Country            0
Direction          0
UNIT               0
VALUE              4
dtype: int64

In [30]:
# Checking against some malform data - get reference about malform data
mal = ["n.a", "?", "NA", "n/a", "na", "--"]
filtered_df = pd.read_csv("airandtravel1.csv" , na_values=mal)

In [31]:
# No malform data recognised as result is the same as above. 
filtered_df.isnull().sum()

STATISTIC Label    0
Year               0
Month              0
Country            0
Direction          0
UNIT               0
VALUE              4
dtype: int64

In [32]:
mean_value = filtered_df['VALUE'].mean()

# Fill NaN values with the mean of VALUE - give reference in report as to why this is important
filtered_df['VALUE'].fillna(mean_value, inplace=True)

In [33]:
# Now my dataset is clean and there are no NaN or missing values
filtered_df.isnull().sum()

STATISTIC Label    0
Year               0
Month              0
Country            0
Direction          0
UNIT               0
VALUE              0
dtype: int64